In [1]:
import numpy as np
%matplotlib widget
import gillespy2
from gillespy2 import Model, Species, Reaction, Parameter, RateRule, AssignmentRule, FunctionDefinition
from gillespy2 import EventAssignment, EventTrigger, Event

In [2]:
class Dimerization(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Dimerization")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="k_c", expression=0.005))
        self.add_parameter(Parameter(name="k_d", expression=0.08))

        # Species
        self.add_species(Species(name="Monomer", initial_value=30, mode="discrete"))
        self.add_species(Species(name="Dimer", initial_value=0, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="r_creation", reactants={'Monomer': 2}, products={'Dimer': 1}, rate=self.listOfParameters["k_c"]))
        self.add_reaction(Reaction(name="r_dissociation", reactants={'Dimer': 1}, products={'Monomer': 2}, rate=self.listOfParameters["k_d"]))

        # Timespan
        self.timespan(np.linspace(0, 100, 101))

In [3]:
model = Dimerization()

In [4]:
from sciope.utilities.gillespy2 import wrapper
settings = {"number_of_trajectories":1}
simulator = wrapper.get_simulator(gillespy_model=model, run_settings=settings, species_of_interest=['Monomer', 'Dimer'])
expression_array = wrapper.get_parameter_expression_array(model)

In [5]:
from dask.distributed import Client
from sciope.designs import latin_hypercube_sampling
from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH

c = Client()
lhc = latin_hypercube_sampling.LatinHypercube(xmin=expression_array, xmax=expression_array*3)
lhc.generate_array(1000) #creates a LHD of size 1000

#will use default minimal set of features
summary_stats = SummariesTSFRESH()

In [6]:
from sciope.stochmet.stochmet import StochMET

met = StochMET(simulator, lhc, summary_stats)

/Users/mgeiger/.pyenv/versions/3.8.2/envs/dash-playtest/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.manifold.t_sne module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.manifold. Anything that cannot be imported from sklearn.manifold is now part of the private API.
  warnings.warn(message, FutureWarning)


In [7]:
met.compute(n_points=50, chunk_size=2)

In [8]:
#First lets add some appropiate information about the model and features
met.data.configurations['listOfParameters'] = list(model.listOfParameters.keys())
met.data.configurations['listOfSpecies'] = list(model.listOfSpecies.keys())
met.data.configurations['listOfSummaries'] = met.summaries.features
met.data.configurations['timepoints'] = model.tspan

In [9]:
# Here we use UMAP for dimension reduction
met.explore(dr_method='umap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Clear all', style=ButtonStyle())

Button(description='Submit', style=ButtonStyle())

IntSlider(value=0, max=10, min=-1)

Dropdown(description='Species plot 1', options={'Monomer': 0, 'Dimer': 1}, value=0)

Dropdown(description='Species plot 2', index=1, options={'Monomer': 0, 'Dimer': 1}, value=1)

,k_c,k_d
0,0.009565,0.089449
1,0.009735,0.213413
2,0.013158,0.188909
3,0.012888,0.162963
4,0.008053,0.157678
5,0.013038,0.100501
6,0.008043,0.150310
7,0.007452,0.226547
8,0.007122,0.149990
9,0.013428,0.221902


In [40]:
help(model_lp.train)

Help on method train in module sciope.models.label_propagation:

train(inputs, targets, min_=0.01, max_=30, niter=10, stepsize=0.1) method of sciope.models.label_propagation.LPModel instance
    Train the LP model given the data
    
    Parameters
    ----------
    inputs : nd-array
        independent variables
    targets : vector
        dependent variable
    min : float
        []
    max : float
        []
    niter : int
        number of training iterations
    stepsize : float
        []



In [10]:
from sciope.models.label_propagation import LPModel
#here lets use the dimension reduction embedding as input data
data = met.dr_model.embedding_

model_lp = LPModel()
#train using basinhopping
model_lp.train(data, met.data.user_labels, min_=0.01, max_=10, niter=50)

/Users/mgeiger/.pyenv/versions/3.8.2/envs/dash-playtest/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)


basinhopping step 0: f 0.0937494
basinhopping step 1: f 0.0932612 trial_f 0.0932612 accepted 1  lowest_f 0.0932612
found new global minimum on step 1 with function value 0.0932612
basinhopping step 2: f 0.0932705 trial_f 0.0932705 accepted 1  lowest_f 0.0932612
basinhopping step 3: f 0.09326 trial_f 0.09326 accepted 1  lowest_f 0.09326
found new global minimum on step 3 with function value 0.09326
basinhopping step 4: f 0.0932612 trial_f 0.0932612 accepted 1  lowest_f 0.09326
basinhopping step 5: f 0.09326 trial_f 0.09326 accepted 1  lowest_f 0.09326
basinhopping step 6: f 0.0932607 trial_f 0.0932607 accepted 1  lowest_f 0.09326
basinhopping step 7: f 0.0933522 trial_f 0.0933522 accepted 1  lowest_f 0.09326
basinhopping step 8: f 0.0933041 trial_f 0.0933041 accepted 1  lowest_f 0.09326
basinhopping step 9: f 0.0934684 trial_f 0.0934684 accepted 1  lowest_f 0.09326
basinhopping step 10: f 0.093261 trial_f 0.093261 accepted 1  lowest_f 0.09326
basinhopping step 11: f 0.0932984 trial_f 0.

In [11]:
# just to vislualize the result we will map the label distribution to the user_labels (will enable us to see the LP model 
# output when using method "explore")

user_labels = np.copy(met.data.user_labels)
#takes the label corresponding to index 0
met.data.user_labels = model_lp.model.label_distributions_[:,0]

In [12]:
met.explore(dr_method='umap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Clear all', style=ButtonStyle())

Button(description='Submit', style=ButtonStyle())

IntSlider(value=0, max=10, min=-1)

Dropdown(description='Species plot 1', options={'Monomer': 0, 'Dimer': 1}, value=0)

Dropdown(description='Species plot 2', index=1, options={'Monomer': 0, 'Dimer': 1}, value=1)

,k_c,k_d
0,0.009565,0.089449
1,0.009735,0.213413
2,0.013158,0.188909
3,0.012888,0.162963
4,0.008053,0.157678
5,0.013038,0.100501
6,0.008043,0.150310
7,0.007452,0.226547
8,0.007122,0.149990
9,0.013428,0.221902


In [44]:
met.data.user_labels = user_labels